In [1]:
import sys
from pathlib import Path

#Path.cwd().parent depends on where you run this code from
#Path(__file__).resolve().parents[1] depends on where this file is located on disk
#
#project_root = .../1099-reconciliation-pipeline
project_root = Path.cwd().parent  # running from notebooks/  folder (cwd = current working directory) - .parent gets us to the folder above
sys.path.append(str(project_root)) #sys.path is a list of folders where python looks for modules
                                    # we add the project root to that list with .append

print(project_root)

/Users/manuelreyes/Desktop/dev/1099_reconciliation_pipeline


In [2]:
from src import load_data
from src.clean_matrix import clean_matrix
from src.age_taxcode_analysis import clean_relius_demo, run_age_taxcode_analysis
from src.build_correction_file import build_correction_dataframe, write_correction_file

from src.config import RAW_DATA_DIR

relius_demo_path = RAW_DATA_DIR / "real_demo_relius_2025.xlsx"
matrix_path = RAW_DATA_DIR / "real_all_matrix_2025.xlsx"

# Load  and clean Matrix raw data
matrix_raw = load_data.load_matrix_excel(path=matrix_path, use_sample_if_none=False)
matrix_clean = clean_matrix(matrix_raw)

# # Load  and clean Relius raw data
relius_demo_clean = clean_relius_demo(relius_demo_path)


#.shape is an attribute of pandas DataFrames that returns a tuple of
# (number of rows, number of columns) - e.g. (1000, 15)
relius_demo_clean.shape

/Users/manuelreyes/Desktop/dev/1099_reconciliation_pipeline/src/clean_matrix.py:101: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  return pd.to_datetime(series, errors="coerce").dt.date


(61584, 6)

In [3]:
relius_demo_clean.head(10)

,plan_id,ssn,first_name,last_name,dob,term_date
2110,100MBD,113741850,Donald,Mangan,1988-11-12,NaT
2197,100MBD,116648243,Deborah,Mosloskie,1966-09-26,NaT
3595,100MBD,145609822,Beth,Conley,1968-03-22,NaT
4751,100MBD,159460981,Paul J,Petrosky,1952-04-12,2013-09-01
5327,100MBD,159667416,William,Delaney,1969-06-21,NaT
6153,100MBD,160643429,Diana,Hanobeck,1973-07-25,NaT
6270,100MBD,160667435,Christina,Miller,1985-06-16,NaT
7795,100MBD,162581348,Yvonne,Benedict,1961-11-12,NaT
7958,100MBD,162642206,Melanie,Ellixson,1975-12-08,NaT
8886,100MBD,163607906,Richard,Borcky,1963-06-18,NaT


In [4]:
relius_demo_clean.info()

<class 'pandas.core.frame.DataFrame'>
Index: 61584 entries, 2110 to 58800
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   plan_id     61584 non-null  object
 1   ssn         61584 non-null  object
 2   first_name  61584 non-null  object
 3   last_name   61584 non-null  object
 4   dob         61527 non-null  object
 5   term_date   12358 non-null  object
dtypes: object(6)
memory usage: 3.3+ MB


In [5]:
age_matches = run_age_taxcode_analysis(matrix_clean, relius_demo_clean)

age_matches.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6980 entries, 0 to 6979
Data columns (total 29 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   plan_id                6980 non-null   object 
 1   ssn                    6980 non-null   object 
 2   participant_name       6970 non-null   object 
 3   state                  6980 non-null   object 
 4   gross_amt              6980 non-null   float64
 5   txn_date               6980 non-null   object 
 6   txn_method             6980 non-null   object 
 7   tax_code_1             6970 non-null   object 
 8   tax_code_2             144 non-null    object 
 9   tax_form               6980 non-null   object 
 10  dist_type              6980 non-null   object 
 11  transaction_id         6980 non-null   object 
 12  matrix_account         6980 non-null   object 
 13  partipant_name         6980 non-null   object 
 14  dob                    6632 non-null   object 
 15  term

In [ ]:
age_matches.head(10)

,plan_id,ssn,participant_name,state,gross_amt,txn_date,txn_method,tax_code_1,tax_code_2,tax_form,...,age_at_distribution,age_at_termination,expected_tax_code_1,expected_tax_code_2,correction_reason,action,match_status,code_matches_expected,suggested_tax_code_1,suggested_tax_code_2
0,IRA127PLAT,194362032,Sandra Marsh,PA,153.22,2025-11-04,ACH Distribution,7,<NA>,1099-R,...,78.25,<NA>,7,<NA>,age_59_5_or_over_normal_distribution,<NA>,perfect_match,True,7,<NA>
1,IRA127PLAT,209501562,Kathryn Clausen,PA,200.00,2025-11-04,ACH Distribution,7,<NA>,1099-R,...,69.916667,<NA>,7,<NA>,age_59_5_or_over_normal_distribution,<NA>,perfect_match,True,7,<NA>
2,IRA127PLAT,194362032,Sandra Marsh,PA,153.22,2025-10-16,ACH Distribution,7,<NA>,1099-R,...,78.25,<NA>,7,<NA>,age_59_5_or_over_normal_distribution,<NA>,perfect_match,True,7,<NA>
3,IRA127PLAT,209501562,Kathryn Clausen,PA,200.00,2025-10-03,ACH Distribution,7,<NA>,1099-R,...,69.833333,<NA>,7,<NA>,age_59_5_or_over_normal_distribution,<NA>,perfect_match,True,7,<NA>
4,IRA127PLAT,194362032,Sandra Marsh,PA,153.22,2025-09-03,ACH Distribution,7,<NA>,1099-R,...,78.083333,<NA>,7,<NA>,age_59_5_or_over_normal_distribution,<NA>,perfect_match,True,7,<NA>
5,IRA127PLAT,209501562,Kathryn Clausen,PA,200.00,2025-09-03,ACH Distribution,7,<NA>,1099-R,...,69.75,<NA>,7,<NA>,age_59_5_or_over_normal_distribution,<NA>,perfect_match,True,7,<NA>
6,IRA127PLAT,194362032,Sandra Marsh,PA,153.22,2025-08-05,ACH Distribution,7,<NA>,1099-R,...,78.0,<NA>,7,<NA>,age_59_5_or_over_normal_distribution,<NA>,perfect_match,True,7,<NA>
7,IRA127PLAT,209501562,Kathryn Clausen,PA,200.00,2025-08-05,ACH Distribution,7,<NA>,1099-R,...,69.666667,<NA>,7,<NA>,age_59_5_or_over_normal_distribution,<NA>,perfect_match,True,7,<NA>
8,IRA127PLAT,194362032,Sandra Marsh,PA,153.22,2025-07-03,ACH Distribution,7,<NA>,1099-R,...,77.916667,<NA>,7,<NA>,age_59_5_or_over_normal_distribution,<NA>,perfect_match,True,7,<NA>
9,IRA127PLAT,209501562,Kathryn Clausen,PA,200.00,2025-07-03,ACH Distribution,7,<NA>,1099-R,...,69.583333,<NA>,7,<NA>,age_59_5_or_over_normal_distribution,<NA>,perfect_match,True,7,<NA>


In [7]:
#Build Excel correction file (same build as inherited corrections engine)
age_correction_df = build_correction_dataframe(age_matches)

age_correction_df.head(10)

,Transaction Id,Transaction Date,Participant SSN,Participant Name,Matrix Account,Current Tax Code 1,Current Tax Code 2,New Tax Code 1,New Tax Code 2,Reason,Action
0,41386906,2025-07-31,161389368,Letitia Leitzel,07P6LM3T,G,<NA>,7,<NA>,age_59_5_or_over_normal_distribution,UPDATE_1099
1,43303313,2025-10-06,161389368,Letitia Leitzel,07P6LM3T,G,<NA>,7,<NA>,age_59_5_or_over_normal_distribution,UPDATE_1099
2,42302481,2025-09-02,182601810,Thomas Richards,07P6LM3T,G,<NA>,7,<NA>,age_59_5_or_over_normal_distribution,UPDATE_1099
3,44178532,2025-11-04,193361668,Robert Klemow,07P6LM3T,G,<NA>,7,<NA>,age_59_5_or_over_normal_distribution,UPDATE_1099
4,40319558,2025-06-23,172448354,BARBARA KUMAR,07P6LM3V,G,<NA>,7,<NA>,age_59_5_or_over_normal_distribution,UPDATE_1099
5,41292023,2025-07-28,181466683,Martha Laux,07P6LM3V,G,<NA>,7,<NA>,age_59_5_or_over_normal_distribution,UPDATE_1099
6,41794288,2025-08-13,181466683,Martha Laux,07P6LM3V,G,<NA>,7,<NA>,age_59_5_or_over_normal_distribution,UPDATE_1099
7,42699653,2025-09-12,160467412,Larry Myers,07P6LM3W,G,<NA>,7,<NA>,age_59_5_or_over_normal_distribution,UPDATE_1099
8,41982024,2025-08-20,170624760,Diana Kristobek,07P6LM3W,G,<NA>,2,<NA>,no_term_date_55_plus_in_2025,UPDATE_1099
9,42785888,2025-09-16,170624760,Diana Kristobek,07P6LM3W,G,<NA>,2,<NA>,no_term_date_55_plus_in_2025,UPDATE_1099


Notes:
- Currently the age_correction DataFrame is finding all discrepancy between G and codes 1, 2 or G.
- Age correction analysis is working, however code G is not based on age but in distribution type (G -> Rollovers).
- We need to filter out distributions that are G (Traditional rollover) and H (Roth rollover) and exclude plans that are inherited since these are always code 4.